In [1]:
import numpy as np
import pandas as pd
import yfinance
import ta

# Base Strategy
The base strategy for a simple stochrsi strategy:
### Part 1: Buying Requirements
  1. The close price must be above the 200 sma (Simple Moving Average)
  2. The Stochastic RSI K must below certain percentage (n%)
  3. Placing a n % limit buy order the next period with an order limit of n periods
### Part 2: Selling Requirements
  1. The trailing stop is equal to n% of the limit buy percentage
  2. Otherwise sell at trailing stop

## Getting Price Data
Getting the price data of symbol using the yfinance library

In [2]:
# Starting with the analysis with AAPL
# download AAPL's daily price from 2010-01-01 to 2020-12-31 (10 years)
symbol = 'AAPL'
ticker = yfinance.Ticker(symbol)
prices = ticker.history(start='2010-01-01', end='2020-12-31')

In [3]:
prices

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-01-04,6.508335,6.540963,6.476316,6.526021,493729600,0.0,0.0
2010-01-05,6.544014,6.574203,6.502847,6.537305,601904800,0.0,0.0
2010-01-06,6.537303,6.563223,6.426610,6.433319,552160000,0.0,0.0
2010-01-07,6.457105,6.464728,6.374771,6.421426,477131200,0.0,0.0
2010-01-08,6.412888,6.464728,6.375076,6.464118,447610800,0.0,0.0
...,...,...,...,...,...,...,...
2020-12-23,130.808961,131.076190,129.443063,129.621231,88223700,0.0,0.0
2020-12-24,129.977546,132.095669,129.759794,130.620895,54930100,0.0,0.0
2020-12-28,132.620253,135.935997,132.145149,135.292648,124486200,0.0,0.0


## Technical Analysis
Using the ta library to do technical analysis

In [4]:
# first getting the 200 SMA (Simple Moving Average)
sma200 = ta.trend.sma_indicator(prices.Close)
sma200

Date
2010-01-04           NaN
2010-01-05           NaN
2010-01-06           NaN
2010-01-07           NaN
2010-01-08           NaN
                 ...    
2020-12-23    125.017940
2020-12-24    125.643974
2020-12-28    126.873772
2020-12-29    127.833031
2020-12-30    128.765896
Name: sma_12, Length: 2768, dtype: float64

In [5]:
# Second getting the stockastic 
stoch = ta.momentum.stochrsi_k(prices.Close, window=10)
stoch

Date
2010-01-04         NaN
2010-01-05         NaN
2010-01-06         NaN
2010-01-07         NaN
2010-01-08         NaN
                ...   
2020-12-23    0.823845
2020-12-24    0.893412
2020-12-28    0.893412
2020-12-29    0.808878
2020-12-30    0.597015
Name: stochrsi_k, Length: 2768, dtype: float64

## Finding the Buy Signal

In [6]:
# checking if close is above 200 sma and stoch < n%
signal = (prices.Close > sma200) & (stoch < .2)
signal

Date
2010-01-04    False
2010-01-05    False
2010-01-06    False
2010-01-07    False
2010-01-08    False
              ...  
2020-12-23    False
2020-12-24    False
2020-12-28    False
2020-12-29    False
2020-12-30    False
Length: 2768, dtype: bool

In [7]:
# Create new df with close price and signal
ndf = pd.concat([prices.Close, signal], axis=1)
ndf.columns = ['Close', 'Signal']
ndf

,Close,Signal
Date,,
2010-01-04,6.526021,False
2010-01-05,6.537305,False
2010-01-06,6.433319,False
2010-01-07,6.421426,False
2010-01-08,6.464118,False
...,...,...
2020-12-23,129.621231,False
2020-12-24,130.620895,False
2020-12-28,135.292648,False


In [8]:
# Limit order when there is a signal at 97% of the current close price
buy_limit_price = ndf.loc[ndf.Signal, 'Close'] * .90
buy_limit_price

Date
2010-03-22     6.168173
2010-04-20     6.712673
2010-04-29     7.372721
2010-04-30     7.165508
2010-05-03     7.309867
                ...    
2020-01-24    70.280653
2020-01-28    70.143750
2020-02-04    70.399876
2020-06-15    76.115046
2020-07-21    86.103500
Name: Close, Length: 91, dtype: float64

In [9]:
# Shifting the index of the buy price by 1
idx = pd.Series(np.arange(len(ndf)), index=ndf.index)
idx = idx.shift(-1).dropna().loc[ndf.Signal].astype(int)
idx

Date
2010-03-22      54
2010-04-20      74
2010-04-29      81
2010-04-30      82
2010-05-03      83
              ... 
2020-01-24    2532
2020-01-28    2534
2020-02-04    2539
2020-06-15    2630
2020-07-21    2655
Length: 91, dtype: int32

# Back Test

In [10]:
# loop through signal and check the next 10 days 
# buy when the price is meet
# for stocks with good momemtum 
# sell with trailing stop of n% of the drop percetage (in this case 10%)
trail_stop_pct = 1 - .1 * .6
wins = []
num_of_trades = 0
max_buy_period = 20
current_i = 0
for i, bp in zip(idx, buy_limit_price):
  # Check if still holding 
  if current_i >= i:
    continue
  s, e = i, i + max_buy_period
  data = prices.iloc[i: e, 1: 4]
  # Check if the price is lower than buy_limit_price
  has_brought = (data < bp).sum(1)
  if has_brought.sum() == 0:
    # if not brought continue
    continue
  # Since brought moving to sell logic
  buy_date = has_brought.loc[(has_brought > 0)].index[0]
  sect_data = prices.loc[buy_date:, ['Open', 'High', 'Low', 'Close']]
  # Second loop 
  # looping through all prices to find selling date
  stop_price = sect_data.High.iloc[0] * trail_stop_pct
  current_high = sect_data.High.iloc[0]
  for ii, (d, row) in enumerate(sect_data.iloc[1:].iterrows()):
    # check sell
    is_sell = False
    if row.Open < stop_price:
      # if open prices is less than stop limit sell
      wp = row.Open / bp - 1
      is_sell = True
    elif row.Low < stop_price:
      # if stop limit is triggered sell
      wp = stop_price / bp - 1
      is_sell = True
    elif ii+2 == len(data):
      print(i, len(data))
      wp = row.Close / bp - 1
      is_sell = True

    if is_sell:
      # Sell logic
      wins.append(wp)
      current_i = ii + 1 + i
      num_of_trades += 1
      break

    # Increasing the stop_price when there is a new high
    if current_high < row.High:
      stop_price = row.High * stop_price

In [11]:
# Checking the strategy gain
gains = np.array(wins)
strategy_gain = np.prod(gains + 1) - 1
print(f'This Strategy has a gaining rate of : {strategy_gain*100:.2f}%')
print(f'Total trades of {num_of_trades}.') 
win_rate = len(gains[gains > 0]) / len(gains)
avg_win_pct = gains[gains > 0].mean() if win_rate != 0 else 0
biggest_win = gains[gains > 0].max() if win_rate != 0 else 0
avg_lost_pct = gains[gains < 0].mean() if win_rate != 1 else 0
biggest_lost = gains[gains < 0].min() if win_rate != 1 else 0

print(f'''Strategy has a win rate of {win_rate*100:.2f}% 
with an average win percentage of {avg_win_pct*100:.2f}% 
with a biggest win of {biggest_win*100:.2f}%
with an average lost percentage of {avg_lost_pct*100:.2f}% 
with a biggest lost of {biggest_lost*100:.2f}%''')

This Strategy has a gaining rate of : 47.91%
Total trades of 12.
Strategy has a win rate of 83.33% 
with an average win percentage of 4.80% 
with a biggest win of 10.33%
with an average lost percentage of -3.47% 
with a biggest lost of -5.12%


In [12]:
# Seeing how much can be gain through moment
static_gain = prices.Close[-1] / prices.Close[0] - 1
print(f'The static gain of {symbol} is :  {static_gain*100:.2f}%') 

The static gain of AAPL is :  1928.08%


In [13]:
# Seeing how much can be gain when sell at the highest price
highest_gain = prices.High.max() / prices.Close[0] - 1
print(f'The maximum gain of {symbol} is :  {highest_gain*100:.2f}%') 

The maximum gain of AAPL is :  2004.98%


The result does not seem to be ideal. Let see how can we improve it.